In [8]:
import requests
from bs4 import BeautifulSoup

r = requests.get("https://www.century21.com/real-estate/los-angeles-ca/LCCALOSANGELES/", 
                 headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c = r.content
soup = BeautifulSoup(c, "html.parser")

card_body = soup.find_all("div", {"class", "card-body"})

page_num = soup.find_all("a", {"class", "page-link"})[2].text

l = []
base_url = "https://www.century21.com/real-estate/los-angeles-ca/LCCALOSANGELES/?s="
for page in range(0, int(page_num)*24, 24):
    # print(base_url + str(page))
    r = requests.get(base_url + str(page))
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    card_body = soup.find_all("div", {"class", "card-body"})
    
    for item in card_body:
        d = {}
        
        d["Address"] = item.find("span").text
        
        try:
            d["Locality"] = item.find("strong").text
        except:
            d["Locality"] = None
            
        d["Price"] = item.find("div", {"class", "property-price"}).text.replace("\n", "",).replace(" ", "")
        
        try:
            d["Beds"] = item.find("div", {"class", "beds"}).find("span").text
        except:
            d["Beds"] = None
    
        try:
            d["Area"] = item.find("div", {"class", "square-footage"}).find("span").text
        except:
            d["Area"] = None
    
        try:
            d["Full Baths"] = item.find("span", {"class", "full-baths"}).text
        except:
            d["Full Baths"] = None
    
        try:
            d["Half Baths"] = item.find("span", {"class", "half-baths"}).text
        except:
            d["Half Baths"] = None
    
        try:
            for feature in soup.find("ul", {"class", "list-table"}):
                print(feature)
                if "Lot Size" in feature:
                    d["Lot Size"] = feature.find("div", {"class":"cell_2"}).text
        except:
            d["Lot Size"] = None
        l.append(d)

import pandas
df = pandas.DataFrame(l)
# df

df.to_csv("Output.csv")